In [29]:
import carla
import time

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

def spawn_vehicle(vehicle_index=0, spawn_index=0, x_offset=0, y_offset=0, pattern='vehicle.*model3*') -> carla.Vehicle:
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    spawn_point.location.x += x_offset
    spawn_point.location.y += y_offset
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

# Spawn target vehicle
target_vehicle = spawn_vehicle(x_offset=-50)

time.sleep(5)

# Spawn the vehicle
ego_vehicle = spawn_vehicle()

# Add the radar sensor
radar_bp = world.get_blueprint_library().find('sensor.other.radar')
radar_bp.set_attribute('horizontal_fov', '10')  # Horizontal field of view
radar_bp.set_attribute('vertical_fov', '10')    # Vertical field of view
radar_bp.set_attribute('range', '20')           # Maximum range

radar_transform = carla.Transform(carla.Location(x=2.0, z=1.0))
radar = world.spawn_actor(radar_bp, radar_transform, attach_to=ego_vehicle)

# Variable to store the minimum TTC
min_ttc = float('inf')

def radar_callback(data: carla.RadarMeasurement):
    global min_ttc, min_distance
    min_ttc = float('inf')

    for detection, i in zip(data, range(len(data))):
        absolute_speed = abs(detection.velocity)

        # Calculate TTC
        if absolute_speed != 0:
            ttc = detection.depth / absolute_speed
            print(ttc)
            if ttc < min_ttc:
                min_ttc = ttc

# Register the radar callback
radar.listen(radar_callback)
i=0
try:
    while True:
        # TTC threshold (e.g., 2 seconds)
        ttc_threshold = 1.0

        # Braking control
        if min_ttc < ttc_threshold:
            control = carla.VehicleControl()
            control.brake = 1.0  # Maximum braking
            ego_vehicle.apply_control(control)
            print("Emergency braking activated!")
            i += 1
            if i>10:
                break
        else:
            control = carla.VehicleControl()
            control.throttle = 0.5  # Maintain constant speed
            ego_vehicle.apply_control(control)
        
        time.sleep(0.05)
    time.sleep(2)
except KeyboardInterrupt:
    print("Keyboard interrupt detected.")
finally:
    radar.stop()
    radar.destroy()
    ego_vehicle.destroy()
    target_vehicle.destroy()


0.0028093880236608844
248.3417582011587
118.86612713094378
126.50765859695022
167.44965877512868
146.06959326484437
153.0232713140095
97.47497296147726
187.23601776713514
103.3748513004226
225.4703759939906
283.3034531518753
153.67620280535485
148.18127983319528
277.76123657276764
246.82025017298523
233.77130645987538
111.27688568936294
383.6263706109834
282.44861985186157
342.9963316792883
242.84065955143916
497.5912637058054
604.0962826760413
859.0918948847312
1154.5720779402982
870.8957117504382
926.1372458451699
464.64910857124795
2262.4289980830067
2750.5720965663886
5079.730255804232
3341.245556172824
4539.777458685668
6487.744133759857
3451.7619701642093
14942.619684180381
19356.852785300034
21577.539815739274
9113.378291064399
3361.0834115769094
6779.923553960936
4676.874281431083
4936.914503595271
4421.548617194773
3832.1693607783027
3216.419431861688
2261.904437420112
3905.283623438451
2100.417410262298
1554.968994947963
1682.9185051997752
2490.298325437178
2071.314478195897
